In [2]:
import pandas as pd

In [11]:
data = pd.read_csv('Estadísticas_de_ejecución_de_los_gastos_por_institución-1.csv',  encoding='latin-1')

In [26]:
# data.columns
data.rename(columns={
    'CAPITULO':'CAPITULO',
    'SUBCAPITULO':'SUBCAPITULO',
    'UNIDAD EJECUTORA':'UNIDAD_EJECUTORA',
    'PROGRAMA':'PROGRAMA',
    'SUBPROGRAMA':'SUBPROGRAMA',
    'PROYECTO':'PROYECTO',
    'ACTIVIDAD/OBRA':'ACTIVIDAD_OBRA',
    'PRESUPUESTO INICIAL':'PRESUPUESTO_INICIAL',
    'PRESUPUESTO VIGENTE':'PRESUPUESTO_VIGENTE',
    'DEVENGADO ':'DEVENGADO',
    'MES':'MES',
    'AÑO':'ANIO'

}, inplace=True)

data

In [62]:
import duckdb
import json
import copy

In [41]:
capitulo =  duckdb.query("SELECT CAPITULO,  SUM(PRESUPUESTO_INICIAL) INICIAL, SUM(PRESUPUESTO_VIGENTE) VIGENTE, SUM(DEVENGADO) DEVENGADO FROM data WHERE ANIO = 2023 GROUP BY CAPITULO ").df()
subcap = duckdb.query("SELECT CAPITULO, SUBCAPITULO,  SUM(PRESUPUESTO_INICIAL) INICIAL, SUM(PRESUPUESTO_VIGENTE) VIGENTE, SUM(DEVENGADO) DEVENGADO FROM data WHERE ANIO = 2023 GROUP BY CAPITULO, SUBCAPITULO").df()
ue = duckdb.query("SELECT CAPITULO, SUBCAPITULO, UNIDAD_EJECUTORA,  SUM(PRESUPUESTO_INICIAL) INICIAL, SUM(PRESUPUESTO_VIGENTE) VIGENTE, SUM(DEVENGADO) DEVENGADO FROM data WHERE ANIO = 2023 GROUP BY CAPITULO, SUBCAPITULO, UNIDAD_EJECUTORA").df()

In [80]:
# capitulo.to_json('capitulo.json', orient='records')
nivel0  =  json.loads(capitulo.to_json(orient='records'))
nivel1  =  json.loads(subcap.to_json(orient='records'))
nivel2  =  json.loads(ue.to_json(orient='records'))
data  = copy.deepcopy(nivel0)

In [87]:
VALUE_COLUMN = 'INICIAL'

for item in data:
    item['children'] =  [x for x in nivel1 if x['CAPITULO'] == item['CAPITULO']]
    item['name'] = item['CAPITULO']
    item['value'] = item[VALUE_COLUMN]

for item in nivel1:
    item['children'] =  [x for x in nivel2 if x['SUBCAPITULO'] == item['SUBCAPITULO']]
    item['name'] = item['SUBCAPITULO']
    item['value'] = item[VALUE_COLUMN]

for item in nivel2:
    item['name'] = item['UNIDAD_EJECUTORA']
    item['value'] = item[VALUE_COLUMN]

In [86]:
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)  # 'indent' argument makes the file more readable 